In [1]:
import pandas as pd
from scipy import sparse
from scipy.io import mmwrite
import numpy as np
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder

import json
import pickle
import math

In [2]:
df_members = pd.read_csv('data/raw/members.csv')
df_songs = pd.read_csv('data/raw/songs.csv')
df_train = pd.read_csv('data/raw/train.csv')
df_test = pd.read_csv('data/raw/test.csv')                  

In [3]:
def squish_and_discretize(x, low, high, increment, default):
    if math.isnan(x):
        return default
    x = np.clip(x, low, high)
    return str(int(x / increment))

def id_to_index(ids):
    " return a dict like {id: idx} "
    unique_ids = set(ids)
    return dict(zip(unique_ids, np.arange(0, len(unique_ids))))

def rescale_y(y):
    " 0/1 to -1/1 scale "
    return 2 * y - 1

# 0. Extreme sparsity

In [4]:
len(df_train) / (df_members['msno'].nunique() * df_songs['song_id'].nunique())

9.338468694282741e-05

# 1. [Members](https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data)
members.csv

user information.

* msno
* city
* bd: age. Note: this column has outlier values, please use your judgement.
* gender
* registered_via: registration method
* registration_init_time: format %Y%m%d
* expiration_date: format %Y%m%d

In [5]:
Counter(df_members['city'])

Counter({1: 19445,
         3: 204,
         4: 1732,
         5: 2634,
         6: 913,
         7: 93,
         8: 289,
         9: 309,
         10: 216,
         11: 285,
         12: 491,
         13: 3395,
         14: 708,
         15: 1525,
         16: 35,
         17: 152,
         18: 259,
         19: 11,
         20: 27,
         21: 213,
         22: 1467})

In [6]:
Counter(df_members['bd'])
# lots of crazy values
np.median(df_members.query('bd > 15 and bd < 60')['bd'])
df_members['bd_clean'] = df_members['bd'].apply(lambda x: squish_and_discretize(x, 15, 60, 5, 27))
Counter(df_members['bd_clean'])

Counter({'10': 272,
         '11': 116,
         '12': 126,
         '3': 21672,
         '4': 3519,
         '5': 3534,
         '6': 2379,
         '7': 1541,
         '8': 788,
         '9': 456})

In [7]:
print(Counter(df_members['gender']))
df_members.fillna('unknown', inplace=True)
print(Counter(df_members['gender']))

Counter({nan: 19902, 'male': 7405, 'female': 7096})
Counter({'unknown': 19902, 'male': 7405, 'female': 7096})


In [8]:
Counter(df_members['registered_via'])

Counter({3: 4879, 4: 11392, 7: 9433, 9: 8628, 13: 70, 16: 1})

### ignore signup/expiration for now

In [9]:
x_members = ['city', 'bd', 'gender', 'registered_via']
for x in x_members:
    df_members[x] = df_members[x].astype(str)

Are all members in the test set included in this dataset?

In [10]:
df_test['msno'].isin(df_members.msno).all()

True

In [11]:
member_map = id_to_index(df_members['msno'])

In [12]:
member_features = df_members[x_members + ['msno']].\
drop_duplicates()
member_features['member_idx'] = member_features['msno'].apply(lambda x: member_map[x])
member_features.sort_values('member_idx', inplace=True)
member_features['member_idx'] = member_features['member_idx'].astype(str)
assert len(member_features) == len(df_members)

member_X = DictVectorizer().fit_transform(member_features.T.to_dict().values())
mmwrite('data/processed/member_X', member_X)

# 2. Songs
The songs. Note that data is in unicode.

* song_id
* song_length: in ms
* genre_ids: genre category. Some songs have multiple genres and they are separated by |
* artist_name
* composer
* lyricist
* language

In [13]:
song_map = id_to_index(df_songs['song_id'])
df_songs['song_idx'] = df_songs['song_id'].apply(lambda x: song_map[x])

In [14]:
# song length in minutes
print("%d nulls" % df_songs['song_length'].isnull().sum())
df_songs['song_length'].apply(lambda x: x/60 * 1e-3).describe()

0 nulls


count    2.296320e+06
mean     4.116558e+00
std      2.682000e+00
min      3.083333e-03
25%      3.060000e+00
50%      3.777117e+00
75%      4.621150e+00
max      2.028975e+02
Name: song_length, dtype: float64

In [15]:
df_songs['song_length_c'] = df_songs['song_length'].apply(lambda x: squish_and_discretize(x/60000, 1, 8, increment=0.5, default=4))
Counter(df_songs['song_length_c'])

Counter({'10': 124486,
         '11': 77677,
         '12': 54174,
         '13': 40265,
         '14': 29454,
         '15': 20350,
         '16': 77200,
         '2': 80670,
         '3': 72554,
         '4': 145783,
         '5': 236508,
         '6': 373462,
         '7': 422276,
         '8': 329447,
         '9': 212014})

In [16]:
df_songs[['artist_name', 'composer', 'lyricist', 'language']].isnull().mean()

artist_name    0.000000e+00
composer       4.665500e-01
lyricist       8.471132e-01
language       4.354794e-07
dtype: float64

Use artist name, skip composer and lyricist

In [17]:
print(Counter(df_songs['language']))
df_songs['language_c'] = df_songs['language'].fillna(52).astype(int).astype(str)
print(Counter(df_songs['language_c']))

Counter({52.0: 1336694, -1.0: 639467, 3.0: 106295, 17.0: 92518, 24.0: 41744, 31.0: 39201, 10.0: 15482, 45.0: 14435, 59.0: 8098, 38.0: 2385, nan: 1})
Counter({'52': 1336695, '-1': 639467, '3': 106295, '17': 92518, '24': 41744, '31': 39201, '10': 15482, '45': 14435, '59': 8098, '38': 2385})


### genres
there are 200 of them, and they are formatted in an annoying way

In [18]:
df_songs[['song_idx', 'genre_ids']].sample(n=10)

,song_idx,genre_ids
1114387,983847,2122
255412,2248864,1259
1229332,1858820,958
634198,1215,388|940
1742163,1428712,465
2183653,876323,465
60936,1609905,958
1093382,136664,958
780539,965554,947
2207874,1148536,958


In [19]:
%%time
df_genres = df_songs['genre_ids'].apply(lambda s: pd.Series(str(s).split('|')))

CPU times: user 7min 23s, sys: 23.8 s, total: 7min 47s
Wall time: 7min 50s


In [20]:
df_genres['song_idx'] = df_songs['song_idx']

In [21]:
df_genres2 = pd.melt(df_genres, 'song_idx', value_name='genre').\
drop('variable', axis=1).\
dropna().\
sort_values('song_idx')
df_genres2.query('song_idx == 830506')

,song_idx,genre
697615,830506,921


In [22]:
genre_map = id_to_index(df_genres2.genre)
df_genres2['genre_idx'] = df_genres2['genre'].apply(lambda g: genre_map[g])

In [23]:
X_genres = sparse.coo_matrix((np.ones(len(df_genres2)),
                              (df_genres2['song_idx'], df_genres2['genre_idx'])))
assert X_genres.shape[0] == df_songs.song_id.nunique()
X_genres.shape

(2296320, 192)

In [24]:
x_songs = ['song_length_c', 'artist_name', 'language_c']
for x in x_songs:
    df_songs[x] = df_songs[x].astype(str)

In [25]:
%%time
song_features = df_songs[x_songs + ['song_id']].\
drop_duplicates()
song_features['song_idx'] = song_features['song_id'].apply(lambda x: song_map[x])
song_features.sort_values('song_idx', inplace=True)
song_features['song_idx'] = song_features['song_idx'].astype(str)
assert len(song_features) == len(df_songs)



CPU times: user 10.4 s, sys: 680 ms, total: 11.1 s
Wall time: 11.4 s


In [26]:
%%time
song_X = DictVectorizer().fit_transform(song_features.T.to_dict().values())

CPU times: user 3min 31s, sys: 9.31 s, total: 3min 40s
Wall time: 3min 40s


In [27]:
song_X.shape

(2296320, 4815028)

In [28]:
X_genres.shape

(2296320, 192)

In [29]:
sparse.hstack([song_X, X_genres]).shape

(2296320, 4815220)

In [30]:
mmwrite('data/processed/song_X', sparse.hstack([song_X, X_genres]))

In [31]:
# write maps to disk
pickle.dump(genre_map, open('data/processed/genre_map.p', "wb" ))
pickle.dump(song_map, open('data/processed/song_map.p', "wb" ))
pickle.dump(member_map, open('data/processed/member_map.p', "wb"))

Are all the songs in the train and test sets present in df_song?

In [32]:
df_train['song_id'].isin(df_songs['song_id']).mean()

0.99998454743922605

In [33]:
df_test['song_id'].isin(df_songs['song_id']).mean()

0.99999022211444821

In [34]:
# for training, exclude unknown songs
df_train = df_train[df_train['song_id'].isin(df_songs['song_id'])]

# TODO
add these song ids later on

In [35]:
# for training, exclude unknown songs
df_test = df_test[df_test['song_id'].isin(df_songs['song_id'])]

# 3. train, test data
ignore source data for now
# TODO: 
create validation set from training

In [36]:
Counter(df_train.target)

Counter({0: 3662694, 1: 3714610})

In [37]:
len(df_test) / len(df_train)

0.3465717286423333

In [38]:
train_val_split = int(round(2/3 * len(df_train)))
starting_rows = len(df_train)
df_val = df_train.loc[train_val_split:, :]
df_train = df_train.loc[:train_val_split, :]

In [39]:
len(df_train) + len(df_val), starting_rows

(7377305, 7377304)

In [40]:
df_train['member_idx'] = df_train['msno'].apply(lambda x: member_map[x]).astype(str)
df_val['member_idx'] = df_val['msno'].apply(lambda x: member_map[x]).astype(str)
df_test['member_idx'] = df_test['msno'].apply(lambda x: member_map[x]).astype(str)
df_train['song_idx'] = df_train['song_id'].apply(lambda x: song_map[x]).astype(str)
df_val['song_idx'] = df_val['song_id'].apply(lambda x: song_map[x]).astype(str)
df_test['song_idx'] = df_test['song_id'].apply(lambda x: song_map[x]).astype(str)

In [41]:
dims = (len(df_members), len(df_songs))
X_train = sparse.coo_matrix((df_train['target'].apply(rescale_y),
                             (df_train['member_idx'], df_train['song_idx'])),
                            shape=dims)
X_val = sparse.coo_matrix((df_val['target'].apply(rescale_y),
                            (df_val['member_idx'], df_val['song_idx'])),
                           shape=dims)
mmwrite('data/processed/X_train', X_train)
mmwrite('data/processed/X_validation', X_val)